In [1]:
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text
import urllib
import pyodbc
import os
from urllib.parse import quote_plus
from itertools import zip_longest
import re
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#conexao com o BD
driver = os.getenv()
server = os.getenv()
database = os.getenv()
usuario = os.getenv()
password = os.getenv()

def get_connection() -> URL:
    connection_string = """{};SERVER={};DATABASE={};UID={};PWD={}""".format(driver,server,database,usuario,password)
    url_db = quote_plus(connection_string)
    connection_url = f'mssql+pyodbc:///?odbc_connect=+{url_db}'
    return create_engine(connection_url,fast_executemany=True)

In [3]:

def vendas_hausz():
    lista_dicts = []
    engine = get_connection()
    with engine.connect() as conn:
        query_etapas_flexy = (text("""
           SELECT brand.Marca,iflexy.[SKU],basico.NomeProduto,iflexy.[CodigoPedido]
            ,iflexy.[Quantidade]
            ,iflexy.[QtdCaixa],iflexy.[PrecoUnitario],pflexy.StatusPedido
            ,format(iflexy.[DataInserido], 'd', 'pt-BR') as datapedido

            FROM [HauszMapa].[Pedidos].[ItensFlexy]as iflexy
            join [HauszMapa].[Produtos].[ProdutoBasico] as basico
            on basico.SKU = iflexy.[SKU]
            join Produtos.Marca as brand
            on brand.IdMarca = basico.IdMarca
            join [HauszMapa].[Pedidos].[PedidoFlexy] as pflexy
            on pflexy.CodigoPedido = iflexy.CodigoPedido

            where format(iflexy.[DataInserido], 'd', 'pt-BR') like '%/2022%' """))

        execqueryetapas = conn.execute(query_etapas_flexy).all()
        for exc in execqueryetapas:
            dict_items = {}
            for keys, values in exc.items():
                dict_items[keys] = values
            lista_dicts.append(dict_items)
    
    return lista_dicts

pedidos = vendas_hausz()

C:\Users\Crawler\AppData\Local\Temp\ipykernel_23304\1221340531.py:24: SADeprecationWarning: The LegacyRow.items() method is deprecated and will be removed in a future release.  Use the Row._mapping attribute, i.e., 'row._mapping.items()'. (deprecated since: 1.4)
  for keys, values in exc.items():


In [ ]:
pedidosdf = pd.DataFrame(pedidos)
needed_columnms = ['Marca', 'SKU', 'NomeProduto', 'CodigoPedido', 'Quantidade', 'QtdCaixa',
       'PrecoUnitario', 'StatusPedido','datapedido']

df_sums = pedidosdf.groupby(['Marca','datapedido','SKU','CodigoPedido','StatusPedido'])[['Quantidade','PrecoUnitario']].sum()
df_sums = df_sums.reset_index()

df_sums.loc[:,'datapedido'] = pd.to_datetime(df_sums.loc[:,'datapedido'])

ultimos_30=(df_sums['datapedido'] > '2022-05-01') & (df_sums['datapedido'] <= '2022-06-14')
datas = df_sums.loc[ultimos_30]

data_excel = pd.read_excel('estoques.xlsx')
unifica = pd.merge(datas,data_excel, left_on='SKU', right_on='Cód. Merc.', how='left')

unifica.to_excel("vendasultimpos302.xlsx")

datas = datas.groupby(by=["Marca",'SKU',"StatusPedido"], as_index=False).size()

datas.to_excel('resumovendas30.xlsx')

ultimos_60 = (df_sums['datapedido'] > '2022-04-01') & (df_sums['datapedido'] <= '2022-06-14')

datas2 = df_sums.loc[ultimos_60]



#datas2 = datas2.groupby(by=["Marca","StatusPedido"], as_index=False).size()
#datas2.to_excel('Resumomarca60.xlsx')




unifica = pd.merge(datas2,data_excel, left_on='SKU', right_on='Cód. Merc.', how='left')

